In [1]:
from IPython.display import display
import mne
import pickle
import os
import pandas as pd
import numpy as np
from components.preprocessing import preprocessing
from components.dataset import Dataset_Builder
from components.stats import rank_mannwhitneyu
from components.utils import load_cache, save_cache


In [2]:
dataset_path = "data"
dataset = Dataset_Builder(dataset_path=dataset_path)\
            .with_sampling_rate(125)\
            .with_marker(start_minute=0, stop_minute=5, segment_second=30)\
            .build()

data, labels, groups = dataset.load_data_all()
bands = {'delta': [1,3],
    'theta': [4,7],
    'alpha': [8,12],
    'beta': [13,30],
    'gamma': [25,43],
    'slow': [4,13],
    'beta_low': [13,17]
    }
log10 = True
data, labels, groups = dataset.load_data_all()
data.shape, labels.shape, groups.shape
data = preprocessing(data=data,sampling_rate=125,bands=bands,log10=log10)
data.shape, labels.shape, groups.shape

((350, 133), (350,), (350,))

In [2]:
load_cache(path='../selection/125-0-5-30-True/forward', filename='rank.pickle')
# save_cache(data=[131], path='../selection/125-0-5-30-True/forward', filename='rank.pickle')

[131,
 17,
 114,
 11,
 128,
 112,
 0,
 73,
 76,
 120,
 70,
 132,
 28,
 105,
 18,
 94,
 29,
 109,
 113,
 46,
 55,
 57,
 74,
 5,
 107,
 117,
 60,
 93,
 71,
 45,
 115,
 119,
 110,
 116,
 3,
 7,
 104,
 38,
 53,
 111,
 89,
 77,
 130,
 15,
 13,
 85,
 16,
 129,
 22,
 92,
 95,
 37,
 72,
 127,
 20,
 35,
 69,
 97,
 96,
 123,
 25,
 65,
 101,
 34,
 23]

# Perform a forward selection

In [10]:
from components.ml import train_model

for i in range(data.shape[0]):
    cv_scores = train_model(X=data[:,i+1], y=labels, groups=groups, cv_result_prefix=f"cv_results/forward_selection_1")
    print(f"{i+1}|10-CV={format(  round(cv_scores.mean(),5), '.5f')}|STD={format(  round(cv_scores.std(),5), '.5f')}")
# n_split=10
# cv = StratifiedGroupKFold(n_splits=n_split, shuffle=True, random_state=42)
# # # https://scikit-learn.org/stable/auto_examples/svm/plot_rbf_parameters.html
# C_range = np.logspace(-2, 10, 7)
# gamma_range = np.logspace(-9, 3, 7)
# tuned_parameters = [
#         # {'kernel': ['linear'], "C": C_range, "max_iter":[1000]},
#         {"kernel": ["rbf"],    "C": C_range, "max_iter":[1000],  "gamma": gamma_range},
#     ]
# grid = GridSearchCV(SVC(), param_grid=tuned_parameters, cv=cv, n_jobs=os.cpu_count(), refit=True, return_train_score=True)
# grid.fit(X=data, y=labels, groups=groups)
# model = grid.best_estimator_


Exception: logging.FileHandler is not exists in the logger name ml

In [5]:
sfs = SequentialFeatureSelector(SVC(kernel='rbf'), n_features_to_select=25, n_jobs=-1)
sfs.fit(data, labels)


SequentialFeatureSelector(estimator=SVC(), n_features_to_select=25, n_jobs=-1)

In [7]:
sfs.get_support()
# sfs.transform(data).shape

array([ True, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
        True,  True, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False,  True,  True,
        True, False,  True, False, False, False, False, False, False,
       False, False, False,  True,  True,  True,  True,  True, False,
       False, False,  True, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,  True, False, False, False, False, False,  True,
        True,  True,  True,  True, False, False, False,  True, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False,  True, False, False, False,
       False, False, False,  True, False, False, False, False, False,
       False, False, False, False,  True, False, False, False,  True,
       False, False,

In [9]:
data[:, np.array([ True, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
        True,  True, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False,  True,  True,
        True, False,  True, False, False, False, False, False, False,
       False, False, False,  True,  True,  True,  True,  True, False,
       False, False,  True, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,  True, False, False, False, False, False,  True,
        True,  True,  True,  True, False, False, False,  True, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False,  True, False, False, False,
       False, False, False,  True, False, False, False, False, False,
       False, False, False, False,  True, False, False, False,  True,
       False, False,  True, False, False, False, False])].shape

(350, 25)

In [17]:
def get_feature_name():
    import itertools
    ch_names = ['Fp1','Fp2','F3','F4','F7','F8','C3','C4','T3','T4','T5','T6','P3','P4','O1','O2']
    band_names = ['delta','theta','alpha','beta','gamma','slow','beta_low','relative']
    product = itertools.product(band_names, ch_names)
    features = [ feature[1] + '_' + feature[0] for feature in product  ]
    features.append('alpha_frontal')
    features.append('alpha_temporal')
    features.append('alpha_asymetries')
    features.append('beta_frontal')
    features.append('beta_temporal')
    return features
feature_name = get_feature_name()

In [18]:
feature_selected = ['F8_delta']


In [35]:
np.array(set([1,2]).union([5]))

array({1, 2, 5}, dtype=object)

In [33]:
feature_to_search = set(feature_name).difference(feature_selected)

In [25]:
for i in feature_to_search:
    print(i)

P4_relative
F8_beta
F3_beta
Fp2_delta
Fp2_beta_low
F7_delta
O1_delta
T6_slow
Fp1_slow
T4_alpha
O2_theta
T4_slow
F7_beta
P4_theta
P4_slow
O2_delta
T6_relative
O1_alpha
F8_gamma
F3_delta
T5_beta
beta_frontal
C3_beta_low
F8_relative
F4_alpha
F4_theta
C4_beta
O1_beta
T3_delta
O2_gamma
F7_theta
T6_beta_low
F3_alpha
T3_alpha
C3_beta
Fp2_beta
P4_beta_low
O1_theta
T6_alpha
Fp1_relative
Fp2_slow
F3_beta_low
T3_relative
T6_beta
F3_gamma
O1_slow
P3_slow
F7_alpha
T5_delta
T5_alpha
T4_delta
P3_theta
F4_gamma
T4_relative
F8_alpha
C4_beta_low
F4_delta
F7_relative
C4_slow
F7_beta_low
P4_beta
F4_relative
O2_beta
O1_gamma
P4_delta
O2_slow
C3_relative
Fp1_theta
T5_theta
T4_beta
C3_alpha
F8_slow
P3_gamma
Fp1_beta_low
P3_delta
P4_gamma
Fp2_relative
F3_theta
T3_beta
Fp1_alpha
C3_theta
O1_relative
F4_slow
F8_theta
O2_relative
T5_gamma
Fp2_theta
T4_gamma
T6_theta
alpha_frontal
P3_beta_low
T3_gamma
T4_theta
T6_gamma
C4_alpha
C3_gamma
P3_beta
T6_delta
alpha_asymetries
beta_temporal
Fp1_beta
P3_alpha
F4_beta
T3_